## Title: Hypothesis Tests for Time Series Model

##### Note:  We do the following tests with reasoning, but cannot make decision simply based on hypothesis test

### 1 Test  for Granger Causality
https://www.rdocumentation.org/packages/lmtest/versions/0.9-36/topics/grangertest

### 2. Diebold-Mariano Test For Predictive Accuracy
https://www.rdocumentation.org/packages/forecast/versions/8.4/topics/dm.test


Author: Yiran Jing

Date: 26-10-2018




In [2]:
library(ggplot2)
library(forecast)
library(vars)
require(ggfortify)
require(ggplot2)
require(forecast)
library(urca)

In [3]:
power = read.csv("../data/shorter_clean_data_26_10_2018.csv") # 17451
train_set= power[ c(1:14542), ]

AR_best_fit = read.csv('../data/SAR_train_data.csv',)
AR_336_fit = read.csv("../data/AR336_train_data.csv")



In [17]:
AR_336_fit_residual = AR_336_fit[,2][337:(length(AR_336_fit[,2]))] - power[,2][337:(length(AR_best_fit[,2]))]

AR_best_fit_residual = AR_best_fit[,2][337:(length(AR_best_fit[,2]))] - power[,2][337:(length(AR_best_fit[,2]))]
RW_one_step_resid = power[,2][336:(length(AR_best_fit[,2])-1)] - power[,2][337:(length(AR_best_fit[,2]))]

### Granger Causality Test
Test For bavriate Granger Causality
##### Motivation:

- Grander Test like **Variale selection**

Granger test on whether to combine variables? Since Granger is just seeing if it adding another variable z would be useful for helping forecast x

i.e. If demand and price/temperature are not **‘cointegrated’**, a regress of them is **‘spurious’** and tells us nothing!

**Cointegration** tell about how in long run, there is a relationship between two non-stationary variables. (In economics, it is like if they can achieve to a long-run equilirium relationship)

To be specific, we do can have cointegrated relationship between two variables, i.e. Regress two non-stationary time series, get I(0) result! 

we are checking to see if including x is useful for predicting y when y's own history is already being used for prediction. That is, do not miss the fact the x has to be useful beyond (or extra to) the own history of y.


##### Details:
Model 1 is the unrestricted model that includes the Granger-causal terms. 

Model 2 is the restricted model where the Granger-causal terms are omitted.

The test is a Wald test that assesses whether using the restricted Model 2 in place of Model 1 makes statistical sense (roughly speaking).

H0: no Granger causality

As Pr(>F)< 0.01, we reject H0. 

In [42]:
data = read.csv("../data/shorter_clean_data_26_10_2018.csv")

In [37]:
test_variales = data[,2:4]  # 3 variales, demand, price, air temp

##### Demand and air temp

In [56]:
gre1 = grangertest(data[,2], data[,3], 48)

In [57]:
gre1

Res.Df,Df,F,Pr(>F)
17306,NA,NA,NA
17354,-48,6.583543,1.13782e-40


##### Demand and price

In [45]:
gre2 =grangertest(data[,2], data[,4], 336)

In [49]:
gre2

Res.Df,Df,F,Pr(>F)
16442,NA,NA,NA
16778,-336,2.47233,3.204935e-42


In [52]:
3.204935e-42 >0.05


[1] FALSE

As these two tests both have quite small p_value, so we reject h0, and say there is ???



###  DM test

##### Motivation:
- DM test **model selection**

The Diebold-Mariano test compares the forecast accuracy of two forecast methods.

The null hypothesis is that the two methods have the same forecast accuracy. 

For alternative="less", the alternative hypothesis is that method 2 is less accurate than method 1. 

method 1 is benchmark, method 2 is our suggested model

In empirical applications it is often the case that two or more time series models are available for forecasting a particular variable of interest.

**Question: Are the forecasts equally good?**

actual values {yt;t = 1,...,T}

two forecasts: {yˆ ;t = 1,...,T} {yˆ ;t = 1,...,T}

https://otexts.org/fpp2/AR.html

### 1. DM Test for RW_one step ahead with AR(336) model

In [21]:
dm.test(RW_one_step_resid,AR_336_fit_residual,
        alternative = "less",
        h=1) # The forecast horizon used in calculating e1 and e2


	Diebold-Mariano Test

data:  RW_one_step_residAR_336_fit_residual
DM = -257.34, Forecast horizon = 1, Loss function power = 2, p-value <
2.2e-16
alternative hypothesis: less


**As p_value close to 0, so reject H0, that is the AR(336) do help to get higher forecast accuracy than RW one step ahead, that is good! as we win benckmark!
**

### 2. DM Test for AR(48) with AR(336) model

In [19]:
dm.test(AR_best_fit_residual,AR_336_fit_residual,
        alternative = "less",
        h=1) # The forecast horizon used in calculating e1 and e2


	Diebold-Mariano Test

data:  AR_best_fit_residualAR_336_fit_residual
DM = -2.0527, Forecast horizon = 1, Loss function power = 2, p-value =
0.02006
alternative hypothesis: less


**As p_value = 0.02 < 0.05 at the 5% level, so reject H0, that is the AR(336) do help to get higher forecast accuracy than AR(41)**

but Note that, if at 1% level, we will cannot reject hupothesis! 

### 3. DM Test for ridge with NN

In [23]:
ridge_res_train = read.csv('../data/residuals_ridge_train_181023.csv',)
nn_res_train = read.csv('../data/residuals_nn_train_181027.csv',)

In [17]:
length(nn_res_train[,1])

[1] 12910

In [18]:
length(ridge_res_train[,1])

[1] 12910

In [70]:
ridge_res_train.index = ridge_res_train['train_index']

In [75]:
nn_res_train.index = ridge_res_train['train_index']

In [76]:
dm.test(ridge_res_train[,3],nn_res_train[,3],
        alternative = "less",
        h=1)


	Diebold-Mariano Test

data:  ridge_res_train[, 3]nn_res_train[, 3]
DM = -7.2224, Forecast horizon = 1, Loss function power = 2, p-value =
2.697e-13
alternative hypothesis: less


As p_value close to 0 < 0.05 at the 5% level, so reject H0, that is the NN do help to get higher forecast accuracy than ridge model! 
but Note that, if at 1% level, we will cannot reject hupothesis!